# Setup

In [7]:
import random

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import scipy.stats as stats
import seaborn as sns
from matplotlib_venn import venn2, venn3
from scipy import sparse

In [8]:
seed = 16
random.seed(seed)
np.random.seed(seed)

#esto es para forzar a plt a poner fondos blancos en las figuras aunque el tema del notebook sea oscuro
#plt.rcParams['axes.facecolor'] = 'white'
#plt.rcParams['figure.facecolor'] = 'white'
cmap = plt.get_cmap("tab10")
pio.templates.default = "seaborn"

sns.set_style("darkgrid", rc={'xtick.bottom': True})

In [56]:
data_processed = "../../../data/processed/"
graph_data = data_processed + "graph_data_nohubs/"
reports = "../../../reports/reports_nohubs/"

graph_node_data = pd.read_csv(graph_data+"nohub_graph_node_data.csv")
graph_edge_data = pd.read_csv(graph_data+"nohub_graph_edge_data.csv")


cols = ["comunidad","tamaño","mean_sim_lsa_0","entropia_0","top_monogram","top_5_monogram","top_monogram_score","top_5_monogram_score"]
infomap_summary = pd.read_pickle(reports+"infomap_summary.pkl")[cols]
louvain_summary = pd.read_pickle(reports+"louvain_summary.pkl")[cols]

In [36]:
def get_cluster_dataframes(graph_node_data):
    tamaños_louvain = graph_node_data.comunidades_louvain.dropna().value_counts()
    tamaños_infomap = graph_node_data.comunidades_infomap.dropna().value_counts()

    infomap_clusters = pd.DataFrame(tamaños_infomap).reset_index().rename(columns={"index":"comunidad","comunidades_infomap":"tamaño"}).astype({"comunidad":"int"})
    louvain_clusters = pd.DataFrame(tamaños_louvain).reset_index().rename(columns={"index":"comunidad","comunidades_louvain":"tamaño"}).astype({"comunidad":"int"})

    return infomap_clusters, louvain_clusters

def get_cluster_nodelists(graph_node_data):
    infomap_list = graph_node_data[["node_index","comunidades_infomap"]].dropna().groupby("comunidades_infomap")["node_index"].apply(list)
    louvain_list = graph_node_data[["node_index","comunidades_louvain"]].dropna().groupby("comunidades_louvain")["node_index"].apply(list)

    return infomap_list, louvain_list

def get_cluster_nodesets(graph_node_data):
    infomap_list = graph_node_data[["node_index","comunidades_infomap"]].dropna().groupby("comunidades_infomap")["node_index"].apply(set)
    louvain_list = graph_node_data[["node_index","comunidades_louvain"]].dropna().groupby("comunidades_louvain")["node_index"].apply(set)

    return infomap_list, louvain_list

In [37]:
clusters_infomap,clusters_louvain = get_cluster_nodesets(graph_node_data)

In [11]:
GDA = nx.read_gml(graph_data+"nohub_gda_network.gml", destringizer=int)

In [15]:
list(GDA.nodes(data=True))[0]

(19599,
 {'node_type': 'disease',
  'node_name': 'Hepatomegaly',
  'node_id': 'C0019209',
  'node_source': 'disgenet'})

# Tablas para test de Fisher

Armo las contingency tables para hacer el test

Conjunto de todos los genes:

In [19]:
all_genes = set(graph_node_data.loc[graph_node_data.node_type == "gene_protein", "node_index"].values)

### Conjuntos de genes asociados a comunidades de enfermedades:

In [39]:
nodos_gda = pd.DataFrame(dict(GDA.nodes(data=True))).T.reset_index().rename(columns={"index":"node_index"})
set_nodos_enfermedad = set(nodos_gda.loc[nodos_gda.node_type == "disease", "node_index"].sort_values().values)

In [41]:
clusters_infomap.apply(lambda x: x&set_nodos_enfermedad)

comunidades_infomap
0.0       {30788, 27751, 31818, 22129, 20754, 20755, 207...
1.0       {19073, 20323, 18501, 25670, 18533, 18534, 207...
2.0       {27491, 31907, 31075, 33449, 31026, 32306, 330...
3.0       {24769, 24165, 28907, 19057, 23508, 26587, 249...
4.0       {24125, 30504, 30856, 19694, 18711, 27032, 205...
                                ...                        
1143.0                                              {28812}
1144.0                                              {21725}
1145.0                                              {30313}
1146.0                                {31273, 31274, 23087}
1147.0                                              {31326}
Name: node_index, Length: 1148, dtype: object

In [49]:
def neighbors_from_list(node_list,G):
    neighbor_lists = [G.neighbors(n) for n in node_list] #list of lists
    unnested_set = {item for sublist in neighbor_lists for item in sublist} #set of nodes in lists
    return unnested_set

def get_cluster_sets(set_nodos_enfermedad,cluster_nodesets,G):
    nodos_en_gda = cluster_nodesets.apply(lambda x: list(x&set_nodos_enfermedad))
    cluster_gene_neighbors = nodos_en_gda.apply(lambda x: neighbors_from_list(x,G)).rename("gene_neighbors")
    return cluster_gene_neighbors


In [51]:
vecinos_infomap = get_cluster_sets(set_nodos_enfermedad,clusters_infomap,GDA)
vecinos_louvain = get_cluster_sets(set_nodos_enfermedad,clusters_louvain,GDA)

Veo un ejemplo

In [60]:
ej = 1106
graph_node_data[graph_node_data.comunidades_infomap == ej]

,node_index,node_id,node_name,node_type,node_source,comunidades_infomap,comunidades_louvain,degree_gda,degree_pp,degree_dd
24730,30948,C3277076,"BERNARD-SOULIER SYNDROME, TYPE A2, AUTOSOMAL D...",disease,disgenet,1106.0,252.0,1.0,0.0,4.0
24733,18713,C0005129,Bernard-Soulier Syndrome,disease,disgenet,1106.0,252.0,3.0,0.0,1.0
24738,28504,C1856448,"Bernard-Soulier Syndrome, Type C",disease,disgenet,1106.0,252.0,1.0,0.0,1.0
35816,28503,C1856447,"Bernard-Soulier Syndrome, Type B",disease,disgenet,1106.0,252.0,0.0,0.0,1.0
35835,30959,C3278148,"BERNARD-SOULIER SYNDROME, TYPE A1",disease,disgenet,1106.0,252.0,0.0,0.0,1.0


In [61]:
graph_node_data.set_index("node_index").loc[list(vecinos_infomap[ej])]

,node_id,node_name,node_type,node_source,comunidades_infomap,comunidades_louvain,degree_gda,degree_pp,degree_dd
node_index,,,,,,,,,
6472,2812,GP1BB,gene_protein,disgenet,NaN,NaN,23.0,24.0,0.0
6475,2815,GP9,gene_protein,disgenet,NaN,NaN,6.0,8.0,0.0
6471,2811,GP1BA,gene_protein,disgenet,NaN,NaN,12.0,18.0,0.0


In [62]:
infomap_summary[infomap_summary.comunidad == ej]

,comunidad,tamaño,mean_sim_lsa_0,entropia_0,top_monogram,top_5_monogram,top_monogram_score,top_5_monogram_score
871,1106,5,0.87,0.34,platelet,"[platelet, bleeding, bs, macrothrombocytopenia...",0.7,"[0.7, 0.42, 0.23, 0.21, 0.19]"


### Conjunto de genes que van a pathways:

Armo red de pathways y proteinas

In [63]:
def attributes_from_pd(G:nx.Graph,df:pd.DataFrame,attributes:dict,indexcol):
    """Dados un grafo G y un dataframe df con atributos de sus nodos, especificamos los atributos
    que queremos agregar a los nodos en un diccionario con formato {nombre_columna:nombre_atributo}. 
    La función arma un diccionario con los atributos y el nombre que le queremos poner, indexado con el identificador de nodo que elegimos 
    y los asigna a los nodos del grafo"""
    for attribute,name in attributes.items():
        nx.set_node_attributes(G,pd.Series(df.set_index(indexcol)[attribute]).to_dict(),name)

In [64]:
protein_gene_edges = graph_edge_data[graph_edge_data.edge_type == "pathway_protein"]
PP = nx.from_pandas_edgelist(protein_gene_edges, source="x_index",target="y_index")

attributes_from_pd(PP, graph_node_data, {"node_name":"node_name","node_type":"node_type","node_id":"node_id"},"node_index")

In [83]:
pathway_nodes = graph_node_data.loc[graph_node_data.node_type == "pathway","node_index"]

In [84]:
pathway_nodes

2        34251
11       35548
58       34359
80       35671
109      35034
         ...  
33558    35265
33605    35260
33614    35152
33622    34551
33752    35310
Name: node_index, Length: 2017, dtype: int64

In [96]:
pathway_sets = {}
for pathway in pathway_nodes:
    vecinos = set(PP.neighbors(pathway))
    pathway_sets[pathway] = vecinos

In [97]:
ejemplo = 34196
genes = list(pathway_sets[ejemplo])
display(graph_node_data.set_index("node_index").loc[ejemplo])
graph_node_data.set_index("node_index").loc[genes]

node_id                          R-HSA-1059683
node_name              Interleukin-6 signaling
node_type                              pathway
node_source                   primekg_REACTOME
comunidades_infomap                        NaN
comunidades_louvain                        NaN
degree_gda                                 0.0
degree_pp                                 11.0
degree_dd                                  0.0
Name: 34196, dtype: object

,node_id,node_name,node_type,node_source,comunidades_infomap,comunidades_louvain,degree_gda,degree_pp,degree_dd
node_index,,,,,,,,,
7872,3570,IL6R,gene_protein,disgenet,NaN,NaN,21.0,20.0,0.0
7873,3572,IL6ST,gene_protein,disgenet,NaN,NaN,36.0,29.0,0.0
8008,3716,JAK1,gene_protein,disgenet,NaN,NaN,6.0,81.0,0.0
8009,3717,JAK2,gene_protein,disgenet,NaN,NaN,53.0,90.0,0.0
16877,867,CBL,gene_protein,disgenet,NaN,NaN,21.0,128.0,0.0
13966,6772,STAT1,gene_protein,disgenet,NaN,NaN,20.0,82.0,0.0
14509,7297,TYK2,gene_protein,disgenet,NaN,NaN,31.0,53.0,0.0
13968,6774,STAT3,gene_protein,disgenet,NaN,NaN,138.0,142.0,0.0
17231,9021,SOCS3,gene_protein,disgenet,NaN,NaN,8.0,43.0,0.0


## Test de Fisher

Veo que pares (cluster,pathway) tienen intersección no nula para testear solo esos.

In [21]:
pairs_to_test = []

for cluster in clusters_infomap:
    for pathway in pathway_nodes:
        set_1 = cluster_sets[cluster]
        set_2 = pathway_sets[pathway]
        shared_nodes = len(set_1&set_2)
        if shared_nodes != 0:
            pairs_to_test.append((cluster,pathway))

In [22]:
len(pairs_to_test)

102365

Veo un ejemplo

In [23]:
pairs_to_test[0]

(220, 34200)

In [28]:
cluster, pathway = pairs_to_test[0]
A = cluster_sets[cluster]
B = pathway_sets[pathway]
A_barra = all_genes - A 
B_barra = all_genes - B
matrix = [[len(A&B), len(A_barra&B)], [len(A&B_barra), len(A_barra&B_barra)]]
odd_ratio, pvalue = stats.fisher_exact(matrix)
print(matrix,odd_ratio,pvalue)

[[1, 29], [60, 17232]] 9.903448275862068 0.1005045675556169


In [29]:
results = {}

for pair in pairs_to_test:
    cluster,pathway = pair
    A = cluster_sets[cluster]
    B = pathway_sets[pathway]
    A_barra = all_genes - A 
    B_barra = all_genes - B
    matrix = [[len(A&B), len(A_barra&B)], [len(A&B_barra), len(A_barra&B_barra)]]

    odd_ratio, pvalue = stats.fisher_exact(matrix)
    results[pair] = {"odd_ratio":odd_ratio, "pvalue":pvalue}

In [35]:
results_df = pd.DataFrame(results).T

In [39]:
results_df.to_csv("../../../reports/analisis_comunidades/test_fisher_pathways_infomap.csv",header=False)

# Exploro ejemplos

In [191]:
aver = pd.DataFrame(results).T

In [283]:
aver.sort_values(by="pvalue")[20:30]

odd_ratio        pvalue
73  35535  22.495738  1.071502e-08
128 35619  24.103290  1.180827e-08
    35567  10.065415  1.570481e-08
272 34251  28.956847  1.964000e-08
595 34860        inf  1.999771e-08
    34859        inf  1.999771e-08
128 34914  30.825323  2.539813e-08
    34757  10.981902  2.745659e-08
    35610  15.745374  3.286554e-08
    36037  27.195622  5.021133e-08

In [288]:
cluster, pathway = 272, 34251
A = cluster_sets[cluster]
B = pathway_sets[pathway]
A_barra = all_genes - A 
B_barra = all_genes - B
matrix = [[len(A&B), len(A_barra&B)], [len(A&B_barra), len(A_barra&B_barra)]]
matrix

[[7, 122], [34, 17159]]

In [289]:
infomap_tfidf_data[infomap_tfidf_data.comunidad == cluster]

,comunidad,tamaño,mean_similarity_mono,mean_similarity_bi,mean_similarity_tri,entropia_1,entropia_2,entropia_3,top_monogram,top_monogram_score,...,top_5_monograms,top_5_bigrams,top_5_trigrams,top_5_monograms_score,top_5_bigrams_score,top_5_trigrams_score,mean_similarity_mono_triu,mean_similarity_bi_triu,mean_similarity_tri_triu,pvalor
69,272,25,0.29,0.1,0.08,0.57,0.55,0.54,thrombocytopenia,0.42,...,"[thrombocytopenia, platelet, purpura, thromboc...","[thrombocytopenic purpura, thrombotic thromboc...","[thrombotic thrombocytopenic purpura, granulom...","[0.42, 0.36, 0.35, 0.32, 0.24]","[0.4, 0.33, 0.2, 0.14, 0.13]","[0.4, 0.13, 0.13, 0.13, 0.13]",0.26,0.07,0.05,0.0


In [290]:
graph_node_data[graph_node_data.comunidades_infomap == cluster]

,node_index,node_id,node_name,node_type,node_source,comunidades_infomap,comunidades_louvain,degree_gda,degree_pp,degree_dd
337,339,10120_8555_14837_8556_30867_14536_12775_10743_...,thrombocytopenia,bert_group,primekg,272.0,3.0,0.0,0.0,18.0
339,341,10122_19740,thrombotic thrombocytopenic purpura,bert_group,primekg,272.0,10.0,0.0,0.0,9.0
3871,3887,18896_43768,thrombocytopenic purpura,bert_group,primekg,272.0,10.0,0.0,0.0,9.0
16713,16775,8558_19098,autoimmune thrombocytopenic,bert_group,primekg,272.0,26.0,0.0,0.0,8.0
20480,20627,C0034155,"Purpura, Thrombotic Thrombocytopenic",disease,disgenet,272.0,10.0,4.0,0.0,1.0
20796,20963,C0040034,Thrombocytopenia,disease,disgenet,272.0,3.0,29.0,0.0,1.0
21448,21654,C0154301,Acquired thrombocytopenia,disease,disgenet,272.0,10.0,0.0,0.0,2.0
22286,22533,C0242584,Autoimmune thrombocytopenia,disease,disgenet,272.0,26.0,0.0,0.0,1.0
22974,23281,C0272126,Evans syndrome,disease,disgenet,272.0,45.0,2.0,0.0,3.0
22988,23298,C0272282,"Thrombocytopenia, cyclic",disease,disgenet,272.0,3.0,0.0,0.0,1.0


In [291]:
graph_node_data[graph_node_data.node_index == pathway]

,node_index,node_id,node_name,node_type,node_source,comunidades_infomap,comunidades_louvain,degree_gda,degree_pp,degree_dd
33486,34251,R-HSA-114608,Platelet degranulation,pathway,primekg_REACTOME,NaN,NaN,0.0,129.0,0.0


In [292]:
graph_node_data.set_index("node_index").loc[list(A)]

,node_id,node_name,node_type,node_source,comunidades_infomap,comunidades_louvain,degree_gda,degree_pp,degree_dd
node_index,,,,,,,,,
2816,1437,CSF2,gene_protein,disgenet,NaN,NaN,103.0,9.0,0.0
14339,7174,TPP2,gene_protein,disgenet,NaN,NaN,1.0,14.0,0.0
14219,7056,THBD,gene_protein,disgenet,NaN,NaN,30.0,5.0,0.0
2830,1440,CSF3,gene_protein,disgenet,NaN,NaN,168.0,5.0,0.0
12559,5795,PTPRJ,gene_protein,disgenet,NaN,NaN,8.0,18.0,0.0
4241,2056,EPO,gene_protein,disgenet,NaN,NaN,90.0,7.0,0.0
10394,51816,ADA2,gene_protein,disgenet,NaN,NaN,9.0,2.0,0.0
7965,3674,ITGA2B,gene_protein,disgenet,NaN,NaN,20.0,25.0,0.0
10655,54205,CYCS,gene_protein,disgenet,NaN,NaN,19.0,26.0,0.0


In [293]:
graph_node_data.set_index("node_index").loc[list(B)]

,node_id,node_name,node_type,node_source,comunidades_infomap,comunidades_louvain,degree_gda,degree_pp,degree_dd
node_index,,,,,,,,,
0,1,A1BG,gene_protein,disgenet,NaN,NaN,2.0,12.0,0.0
10244,5155,PDGFB,gene_protein,disgenet,NaN,NaN,27.0,55.0,0.0
7172,308,ANXA5,gene_protein,disgenet,NaN,NaN,16.0,11.0,0.0
16902,87,ACTN1,gene_protein,disgenet,NaN,NaN,4.0,40.0,0.0
17929,948,CD36,gene_protein,disgenet,NaN,NaN,19.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...
10743,54495,TMX3,gene_protein,hippie,NaN,NaN,0.0,3.0,0.0
4600,2243,FGA,gene_protein,disgenet,NaN,NaN,31.0,40.0,0.0
4601,2244,FGB,gene_protein,disgenet,NaN,NaN,9.0,95.0,0.0
